In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import scipy.spatial.distance as dis
from scipy.ndimage import filters
from scipy.sparse import csgraph
from scipy.sparse import csr_matrix
from scipy import stats
import matplotlib.pyplot as plt, seaborn
from io import StringIO
import cProfile
import os
import logging
import glob

In [3]:
logger = logging.getLogger()
#     CRITICAL
#     ERROR
#     WARNING
#     INFO
#     DEBUG
logging.disable(logging.DEBUG);
logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s', '%Y-%m-%d %H:%M:%S')

DATAVERSION = 2
HotelID = 16639
RATEPLANLEVEL = 0
LOS = 1
PERSONCNT = 2

# HOME_FOLDER = '/Users/xyao/Library/Mobile Documents/com~apple~CloudDocs/JupyterHome/Simplification/'
HOME_FOLDER = './'
INPUT_FOLDER2 = './Data{}/'.format(DATAVERSION)
INPUT_FOLDER = './Data/'
OUTPUT_FOLDER = './Result/MINE2/'
os.chdir(HOME_FOLDER)

Observes = ['CostAmt','PriceAmt','LARAmt','LARMarginAmt','LARTaxesAndFeesAmt']

read_data_rt = pd.read_csv(INPUT_FOLDER+'dbo_RoomType_NoIdent.csv', encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
read_data_rt = read_data_rt[['SKUGroupID','RoomTypeID','ActiveStatusTypeID']]

read_data_rt = read_data_rt.loc[read_data_rt['ActiveStatusTypeID']==2]

read_data_rt.drop(['ActiveStatusTypeID'], axis=1, inplace=True)
logger.debug(read_data_rt.head(10))
read_data_rt = read_data_rt.loc[read_data_rt['SKUGroupID'].isin([HotelID])]

read_data_rp = pd.read_csv(INPUT_FOLDER+'dbo_RatePlan_NoIdent.csv', sep=',', engine='python', header=0).fillna(0)
read_data_rp = read_data_rp.loc[(read_data_rp['ActiveStatusTypeID']==2) \
                                & (read_data_rp['RoomTypeID'].isin(read_data_rt['RoomTypeID']))][['RatePlanID']]

read_data = pd.read_csv(INPUT_FOLDER2+str(HotelID)+'_RatePlanLevelCostPrice.csv.zip', sep=',', engine='python', header=0).fillna(0)
read_data = read_data.loc[read_data['RatePlanID'].isin(read_data_rp['RatePlanID'])]

logger.debug(read_data)

#     RatePlanID,StayDate,RatePlanLevel,PersonCnt,LengthOfStayDayCnt,ActiveStatusTypeID,
#     RatePlanLevelCostPriceLogSeqNbr,CostAmt,PriceAmt,CostCode,ChangeRequestIDOld,
#     SupplierUpdateDate,SupplierUpdateTPID,SupplierUpdateTUID,UpdateDate,SupplierLogSeqNbr,
#     ChangeRequestID,LARAmt,LARMarginAmt,LARTaxesAndFeesAmt

read_data.drop(['ActiveStatusTypeID','RatePlanLevelCostPriceLogSeqNbr','ChangeRequestIDOld'], axis=1, inplace=True)
read_data.drop(['SupplierUpdateDate','SupplierUpdateTPID','SupplierUpdateTUID'], axis=1, inplace=True)
read_data.drop(['UpdateDate','SupplierLogSeqNbr','ChangeRequestID'], axis=1, inplace=True)

read_data=read_data.loc[(read_data['RatePlanLevel']==RATEPLANLEVEL) & (read_data['LengthOfStayDayCnt']==LOS) 
             & (read_data['PersonCnt']==PERSONCNT)]

read_data.drop(['RatePlanLevel','LengthOfStayDayCnt','PersonCnt'], axis=1, inplace=True)

df_cdist = pd.DataFrame()

In [6]:
Observe = 'CostAmt':

read_data['z_score']=stats.zscore(read_data[Observe])
print(read_data.head(20))
read_data=read_data.loc[read_data['z_score'].abs()<=3]
read_data_gp = read_data[['StayDate',Observe,'RatePlanID']].groupby(['RatePlanID'],sort=False)
df_corr = pd.DataFrame()

for name, group in read_data_gp:
    group.reset_index(drop=True,inplace=True)
    df_corr[name]=group.set_index('StayDate')[Observe]

# https://blog.csdn.net/walking_visitor/article/details/85128461
# 默认使用 pearson 相关系数计算方法，但这种方式存在误判
df_corr = df_corr.corr()
np.fill_diagonal(df_corr.values, 0)

#df_corr = df_corr.mask(df_corr<0.95)
#plt.figure(figsize=(18, 7))
#seaborn.heatmap(df_corr, center=0, annot=True, cmap='YlGnBu')

graph = csr_matrix(df_corr >= 0.95)
n, labels = csgraph.connected_components(graph)

output_df = pd.DataFrame(columns=['GroupID', 'RatePlanID'])

print ('{}/{}'.format(n,len(read_data_gp.ngroup())))

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
special_values = ['260281795','260281938','260282083','260282084','260281796']
for i in range(n):

    nodes = df_corr.index[np.where(labels==i)]

    if(nodes.values.isin(special_values)):
        read_data.loc[(read_data['RatePlanID'].isin(special_values))].groupby(['StayDate','RatePlanID']).sum()[Observe].unstack().plot(ax=ax)

    df_cdist = df_cdist.append([[Observe,i,nodes.values]],ignore_index=True)

    fig, ax = plt.subplots(figsize=(18,7))

    read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate','RatePlanID']).sum()[Observe].unstack().plot(ax=ax)


df_cdist.columns = ['Observe','GroupID','Group']

df_cdist.to_csv('{}{}_patterngroup.csv'.format(OUTPUT_FOLDER,HotelID), index=False)

plt.show()
plt.close()

In [ ]:
from sklearn import preprocessing
from pandas.core.common import flatten

XA = df_cdist.loc[df_cdist['Observe']=='CostAmt']['Group'].to_numpy()    

for Observe in Observes:

    if Observe=='CostAmt':
        continue
        
    XB = df_cdist.loc[df_cdist['Observe']==Observe]['Group'].to_numpy()
    
    mlb = preprocessing.MultiLabelBinarizer()

    mlb.fit([flatten(XA),flatten(XB)])

    XA = mlb.transform(XA)
    XB = mlb.transform(XB)

    d = dis.cdist(XA, XB, 'cosine')

    df = pd.DataFrame(d)
    df = df.mask(df<0.5,0)
    
    print (Observe)
    print (df)
    
    #plt.figure(figsize=(18, 7))
    #seaborn.heatmap(df, center=0, annot=True, cmap='YlGnBu')
    
    XA = df_cdist.loc[df_cdist['Observe']=='CostAmt']['Group'].to_numpy()

#plt.show()

# print(mlb.inverse_transform(XA[0].reshape(1,-1)))
# print(mlb.inverse_transform(XB[0].reshape(1,-1)))

# print(mlb.inverse_transform(XA[1].reshape(1,-1)))
# print(mlb.inverse_transform(XB[1].reshape(1,-1)))

print(df)

